# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../starter_code/WeatherPy.csv")
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barroso,-21.1869,-43.9758,11.71,95,24,3.30,BR,1667703670
1,aizpute,56.7167,21.6000,4.57,97,53,2.70,LV,1667703672
2,mataura,-46.1927,168.8643,16.14,43,18,2.96,NZ,1667703673
3,kavaratti,10.5669,72.6420,27.11,78,55,3.63,IN,1667703430
4,hambantota,6.1241,81.1185,25.19,100,40,1.54,LK,1667703675


In [3]:
df_sub = vacation_df.dropna().reset_index(drop=True)
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        583 non-null    object 
 1   Lat         583 non-null    float64
 2   Lng         583 non-null    float64
 3   Max Temp    583 non-null    float64
 4   Humidity    583 non-null    int64  
 5   Cloudiness  583 non-null    int64  
 6   Wind Speed  583 non-null    float64
 7   Country     583 non-null    object 
 8   Date        583 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 41.1+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
vacation_df = vacation_df.dropna()
humidity = vacation_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(vacation_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = df_sub.loc[(df_sub["Max Temp"] > 21) & (df_sub["Max Temp"] < 26) 
                              & (df_sub["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
# narrowed_city_df.reset_index(inplace=True)
# del narrowed_city_df['index']
narrowed_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,tura,25.5198,90.2201,23.92,72,0,0.66,IN,1667703626
32,gabriel leyva solano,25.6667,-108.6417,21.12,46,0,3.39,MX,1667703692
54,doha,25.2867,51.5333,25.99,53,0,3.09,QA,1667703710
58,brandon,27.9378,-82.2859,25.95,88,0,1.79,US,1667703713
62,cabo san lucas,22.8909,-109.9124,24.17,63,0,1.34,MX,1667703591


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# geocoordinates
hotel_coordinates = f"{20.8947}, {-156.4700}"
hotel_search = ""

hotel_radius = 5000
hotel_type = "lodging"

# Store into variable named hotel_df:
hotel_df = vacation_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": hotel_radius,
    "types": hotel_type,
    "key": g_key
}


params["location"] = hotel_coordinates

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

hotel_data = response.json()

print(hotel_data["results"][0]["name"])
print(hotel_data["results"][0]["vicinity"])
print(hotel_data["results"][0]["rating"])

Maui Seaside Hotel
100 West Kaahumanu Avenue, Kahului
3.8


In [ ]:
# # Loop through to identify hotels:
# for index, row in hotel_df.iterrows():
    
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#     lat = row["Lat"]
#     lng = row["Lng"]
    
#     params['location'] = f'{lat},{lng}'
    
#     hotel_data = requests.get(base_url, params=params).json()
    
#     try:
        
#         hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
#     except: 
#         print('Issues with finding a hotel at', row['City'])
#         hotel_df.loc[index, "Hotel Name"] = "NA"
    
# hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
